# Utilizzo dei Widget

I widget sono una soluzione molto comoda per fornire interattività alle nostre procedure di analisi dei dati. Essi si agganciano in maniera molto semplice ad una funzione, tipicamente come decoratori, e possono influenzare la visualizzazione di una tabella di dati o di un grafico.


## Installazione

L'installazione comporta l'aggiunta del pacchetto di backend (estensione server di Jupyter Notebook) nel nostro ambiente di lavoro, la sua eventuale attivazione, e l'aggiunta dell'estensione di front-end a Jupyter Lab.

Il backend usa JavaScript e il rendering viene fatto in HTML all'interni del browser. L'esecuzione di un notebook all'interno di un IDE come `PyCharm` comporta la mancata visualizzazione dei widget stessi.

### Prerequisiti
- gestore dei pacchetti `conda` o `pip`
- `nodejs`
- `jupyter`

### Pip
Se si usa `pip` digitare:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

### Conda
Se si usa `conda` portarsi nell'environment in cui si vuole fare l'installazione e digitare:
```
conda install -c conda-forge ipywidgets
```

`conda install` attiverà l'estensione server automaticamente.


### Jupyter Lab
Digitare:
```
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

## Utilizzo base
Per utilizzare i widget, bisognerà importare `ipywidget` nella nostra applicazione:
```python
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
```

Le funzioni `interact`, `interactive`, `interact_manual` sono tre versioni del comando di interazione con l'utente in ingresso e rispettivamente:
- interazione diretta con output del risultato
- creazione di un widget che poi dev'essere mostrato con la funzione esplicita `display` di `IPython`
- interazione manuale al click di un pulsante

La funzione `fixed` serve a fissare un argomento in ingresso alla funzione cui stiamo applicando l'interattività. In caso contrario saranno generati widget per tutti gli input.

Di seguito riportiamo alcuni esempi di chiamata delle tre funzioni.

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x,y):
    return x+y

interact(f,x=10,y=3);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), IntSlider(value=3, description='y…

Come si vede, `interact` chiama la funzione per cui si vogliono definire gli nput in maniera interattiva e il codice `x=10, y=3` stabilisce i valori di defalut visualizzati per gli input e seleziona il widget `IntSlider` sulla base del fatto che sono stati passati dei valori numerici interi alle variabili.

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display

def f(a, b):
    display(a + b)
    return a+b

w=interactive(f,\
    a=widgets.FloatSlider(min=-3.0,max=3.0,step=0.1,value=0.75,continuous_update=False),\
    b=widgets.FloatSlider(min=-3.0,max=3.0,step=0.1,value=-2.4,continuous_update=False))
display(w)

interactive(children=(FloatSlider(value=0.75, continuous_update=False, description='a', max=3.0, min=-3.0), Fl…

Nel codice precedente, la funzione `interactive` definisce esplicitamente un `FloatSlider` con tutte le sue proprietà passate esplicitamente come `kwargs` (_keyword arguments_) al costruttore.

L'uso di `display` qui è duplice: `display(a, b)` visualizza il risultato all'interno della definizione della funzione `f`, mentre `display(w)` visualizza i due widget e il risultato in output. Si provi a commentare alternativamente le due chiamate.

E' interessante il comportamento del `FloatSlider` con il parametro `continuous_update` impostato rispettivamente a `True` o `False`. Il risultato è che, nel caso `True` i valori cambiano con continuità sullo schermo, mentre in caso contrario il cambiamento si ha solo quando lo slider si ferma.

Si noti che `interactive` costruisce una intera GUI in cui i diversi widget sono inclusi come tupla, di cui fa parte anche un widget di `Output`. Basta richiamare le informazioni su `w` dell'esempio precedente:

In [3]:
w.children

(FloatSlider(value=-1.1, continuous_update=False, description='a', max=3.0, min=-3.0),
 FloatSlider(value=1.3, continuous_update=False, description='b', max=3.0, min=-3.0),
 Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '0.19999999999999996'}, 'metadata': {}},)))

La funzione seguente stampa il suo argomento `i` e la sequenza di tutti i numeri da 0 al valore di `i`, posto che questi o i loro quadrati siano palindromi: la rappresentazione stringa del numero `str(x)` coincide col suo inverso ottenuto tramite slicing da destra a sinistra `str(x)[::-1]`. 

In [4]:
%%time
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

slow_function(1e6)

1000000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002]
CPU times: user 432 ms, sys: 3.97 ms, total: 436 ms
Wall time: 436 ms


`slow_function` è molto lenta come ci dice il magic code IPython `%%time` che riporta i tempi di esecuzione della cella.

L'uso diretto di `interact` o `interactive` causa problemi perché ogni aggiornamento dello slider risulta in una nuova esecuzione di `slow_function` che si sovrappone alla precedente.

In [5]:
from ipywidgets import FloatSlider

def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return


interact_manual(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));

interactive(children=(FloatSlider(value=100000.0, description='i', max=10000000.0, min=100000.0, step=100000.0…

L'uso di `interact_manual` aggiunge un pulsante per l'esecuzione _on demand_ di `slow_function` solo dopo che il widget è stato posizionato correttamente.

Infine, come esemplificato di seguito, `fixed` semplicemente blocca un elemento di input in modo tale da non visualizzare il corrispondente widget e da bloccarne il valore.

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x,y):
    return x+y

interact(f,x=10,y=fixed(3));

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

## Tipologie di Widget

Esistono diverse tipologie di widget direttamente abbinate al tipo di dato su sui operano. Nell'esempio seguente, l'uso di un `dict` genera un drop-dpwn menu in cui le chiavi sono le voci del menu mentre la funzione opera direttamente sui valori associati.

In [7]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def f(x):
    return x


interact(f, x={'mango': 1, 'banana':2});

interactive(children=(Dropdown(description='x', options={'mango': 1, 'banana': 2}, value=1), Output()), _dom_c…

Nell'esempio seguente `interact` è usata come decoratore e i due argomenti generano una checkbox a partire da un tipo booleano e un `FloatSlider` a partire da un numero reale.

In [8]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact
def g(x=True, y=1.0):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

Per una lista completa dei widget si consulti la [documentazione di ipywidget](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)

## Gestione degli eventi, animazione e interazione con i grafici

La gestione degli eventi per i widgets avviene attraverso registrazioni di funzioni di callback che fanno da gestione di eventi. Un `Button` ha il suo evento `on_click` cui basta registrare un gestore per ottenere il comportamento voluto, come nel codice seguente, dove viene definito un semplice comportamento di _toggle_ per visualizzare e nascondere alternativamente un messaggio.

In [9]:
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="Click Me!",button_style='info')
output = widgets.Output()
click_count = 0

display(button, output)

def on_button_clicked(b):
    global click_count 
    click_count = click_count + 1
    with output:
        output.clear_output()
        if click_count%2 !=0:
            print("You clicked me!")
        

button.on_click(on_button_clicked)

Button(button_style='info', description='Click Me!', style=ButtonStyle())

Output()

Nell'esempio seguente si illustra una gestione degli eventi generica, associata ad un widget qualunque. L'evento generico è `observe` che registra un gestore il quale deve avere una _signature_ (nel nostro caso `change`) che è in realtà un `dict` che ci dice cosa andiamo a notificare tramite il gestore quando c'è un cambiamento:

- `type` il tipo di notifica
- `new` il nuovo valore di ciò che è cambiato
- `old` il vecchio valore di ciò che è cambiato
- `name` il nome dell'attributo che è cambiato

`observe` seleziona l'attributo da osservare attraverso il parametro `names`.

Si noti l'uso di `HBox` che, insieme a `VBox` crea allineamenti orizzontali o verticali dei singoli widgets passati come una lista.

In [11]:
import ipywidgets as widgets
from IPython.display import display

caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(widgets.HBox([caption, slider]))

L'interazione con i grafici è certamente la caratteristica più interessante dei widgets. Nell'esempio seguente, la funzione `f` traccia una retta di dato coefficiente angolare ed intercetta i quali sono passati come argomento. Il _magic code_ `%matplotlib inline` imposta il backend per il rendering di `matplotlib` che è quello del default di sistema operativo. La funzione `f` ha al suo interno la creazione del grafico come `matplotlib.pyplot.figure` e utilizza il vettore di valori di ascissa generato da `numpy.linspace`.

L'interattività con il grafico si ottiene, al solito, utilizzando `interactive` cui passiamo i widget associati ai parametri di `f`. E' interessante l'utilizzo della proprietà `layout` a livello di singolo widget e a livello di tutta la GUI `inteactive_plot`. La proprietà `layout` consente di accedere direttamente ad alcune proprietà CSS che condizionano il rendering del widget. Il modello di layout utilizzato è [Flexbox](https://css-tricks.com/snippets/css/a-guide-to-flexbox/) di cui `HBox` e `VBox` sono dei casi particolari con delle caratteristiche già predefinite.

Per ulteriori approfondimenti si veda la [documentazione di ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#).

In [12]:
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=widgets.FloatSlider(min=-2.0, max=2.0, orientation='vertical'), 
                               b=widgets.FloatSlider(min=-3, max=3, step=0.5, orientation='vertical'))

output = interactive_plot.children[-1]
output.layout = widgets.Layout(max_width='500px',margin='50px')
output.layout.border = '1px solid red'
interactive_plot.layout.flex_flow='row nowrap'

interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0, orientation='vertical'), Floa…

Infine, l'esempio seguente mostra l'animazione di un grafico usando il widget `Play`. L'interazione si ottiene collegando gli attributi `value` rispettivamente del player e dello slider utilizzato per controllare la pendenza della retta. Il player procede a passi interi e quindi si fa una scalatura del valore del parametro in ingresso ad `f`. Lo slider è usato solo come mezzo per collegare il player al grafico e non viene visualizzato esplicitamente. 

In [14]:
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interactive, fixed
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    
    # scaliamo il coefficiente angolare
    slope = m / 100.0
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, slope * x + b)
    plt.ylim(-5, 5)
    plt.show()

play = widgets.Play(
    interval=0.100,
    value=0,
    min=-200,
    max=200,
    step=1,
    description="spin the straight line",
    disabled=False
)

interactive_plot = interactive(f,m=widgets.IntSlider(min=-200, max=200),b=fixed(0))

widgets.jslink((play, 'value'), (interactive_plot.children[-2], 'value'))
widgets.VBox([play,interactive_plot.children[-1]])

## Esercizi

1. Si crei un grafico che mostra l'andamento di un polinomio $P(x) = \sum_{i=0}^na_ix^i,\ x \in [-5.0,5.0],\ n \in \mathbb{N}$ in cui ogni coefficiente $a_i$ sia regolabile attraverso un `FloatText`. Si utilizzi un layout dinamico a griglia, con non più di tre widget per riga, in dipendenza dal grado del polinomio, impostato attraverso un proprio `IntSlider` con intervallo da $0$ a $10$. La finestra del grafico sarà in fondo, sotto i widget.